In [1]:
import numpy as np 
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from sklearn.metrics import r2_score
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.ticker import PercentFormatter

In [26]:
# 1/ Training data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv', 
                'nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 'nb_conv_layers', 'nb_bn_layers',
                'nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/training_dataset.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
train_dataset = raw_dataset.copy()
train_dataset.isna().sum()
train_dataset = train_dataset.dropna()
train_dataset.tail()

,Model_name,input_shape,ACET,input_width,input_size,nb_params_conv,nb_params_bn,nb_params_fc,flops,nb_layers,nb_conv_layers,nb_bn_layers,nb_fc_layers,sum_activations,weighted_sum_neurons
1606,basic_model_700_7,"(1, 700, 700, 3)",170.762855,700,1470000,1758116,1920,30401850,196248875920,16,7,4,1,119497648,98010896800
1607,basic_model_320_23,"(1, 320, 320, 3)",169.737357,320,307200,2499430,7992,5700,168678951282,44,23,14,1,298643104,84243523392
1608,basic_model_56_23,"(1, 56, 56, 3)",166.602996,56,9408,28069064,11264,24500,170036087884,43,23,14,1,20701712,84982551552
1609,basic_model_224_33,"(1, 224, 224, 3)",155.082757,224,150528,22242728,11392,202810,132106973640,60,33,19,1,31316236,66016370176
1610,basic_model_600_38,"(600, 600, 3)",150.870509,600,1080000,23154824,9896,2710,274420498147,45,38,8,1,148258606,137108348476


In [46]:
# 1/ Test data :

column_names = ['Model_name', 'input_shape', 'ACET', 'input_width', 'input_size', 'nb_params_conv', 
                'nb_params_bn', 'nb_params_fc', 'flops', 'nb_layers', 'nb_conv_layers', 'nb_bn_layers',
                'nb_fc_layers', 'sum_activations', 'weighted_sum_neurons']
raw_dataset = pd.read_csv('./data/test_dim1.csv', names=column_names,
                      na_values = "?", comment='\t', sep=",", skipinitialspace=True)
test_dataset = raw_dataset.copy()
test_dataset.isna().sum()
test_dataset = test_dataset.dropna()
test_dataset.tail()

,Model_name,input_shape,ACET,input_width,input_size,nb_params_conv,nb_params_bn,nb_params_fc,flops,nb_layers,nb_conv_layers,nb_bn_layers,nb_fc_layers,sum_activations,weighted_sum_neurons
108,EfficientNet-B0,"(512, 512, 3)",21.816206,512,786432,3965532,84032,1281000,4085726980,233,81,49,1,117635172,80094501376
109,EfficientNet-B0,"(720, 720, 3)",40.264151,720,1555200,3965532,84032,1281000,8183072596,233,81,49,1,233556068,161236938752
110,EfficientNet-B4,"(90, 90, 3)",12.510738,90,24300,17423416,250400,1793000,595640786,470,160,96,1,10284484,12812090224
111,EfficientNet-B4,"(512, 512, 3)",60.143659,512,786432,17423416,250400,1793000,15877844034,470,160,96,1,305708296,366198974816
112,EfficientNet-B4,"(720, 720, 3)",111.953988,720,1555200,17423416,250400,1793000,31863479154,470,160,96,1,607426440,737567732576


In [28]:
train_dataset.pop('input_width')
train_dataset.pop('nb_params_fc')
train_dataset.pop('nb_params_bn')
train_dataset.pop('nb_bn_layers')
train_dataset.pop('nb_fc_layers')
train_dataset.pop('nb_conv_layers')

0       28
1       28
2       28
3       28
4       28
        ..
1606     7
1607    23
1608    23
1609    33
1610    38
Name: nb_conv_layers, Length: 1611, dtype: int64

In [47]:
test_dataset.pop('input_width')
test_dataset.pop('nb_params_fc')
test_dataset.pop('nb_params_bn')
test_dataset.pop('nb_bn_layers')
test_dataset.pop('nb_fc_layers')
test_dataset.pop('nb_conv_layers')

0      160
1      160
2      168
3      168
4      200
      ... 
108     81
109     81
110    160
111    160
112    160
Name: nb_conv_layers, Length: 113, dtype: int64

In [30]:
Y = train_dataset.pop('ACET')
X = train_dataset

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.001, random_state=42, shuffle=True)

In [32]:
X_train_model_names = X_train.pop('Model_name')
X_train_input_shapes = X_train.pop('input_shape')

In [33]:
X_test_model_names = X_test.pop('Model_name')
X_test_input_shapes = X_test.pop('input_shape')

In [34]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

In [48]:
Y_e = test_dataset.pop('ACET')
X_e = test_dataset
X_e.tail()
X_test_e_model_names = X_e.pop('Model_name')
X_test_e_input_shapes = X_e.pop('input_shape')
dtest_e = xgb.DMatrix(X_e, label=Y_e)

In [36]:
params = {
    'max_depth':14,
    'min_child_weight': 5,
    'eta':.08,
    'subsample': 1.,
    'colsample_bytree': 1.,
    'gamma' : 0.9,
    'objective':'reg:squarederror',
}

In [37]:
params['eval_metric'] = "mae"
num_boost_round = 1000

In [38]:
print(params)

{'max_depth': 14, 'min_child_weight': 5, 'eta': 0.08, 'subsample': 1.0, 'colsample_bytree': 1.0, 'gamma': 0.9, 'objective': 'reg:squarederror', 'eval_metric': 'mae'}


In [39]:
num_boost_round = 51
best_model = xgb.train(
    params,
    dtrain,
    num_boost_round=num_boost_round,
)

In [40]:
loaded_model = best_model

In [49]:
y_pred = loaded_model.predict(dtest_e)
np_test = Y_e.values
y_test = Y_e.values

In [50]:
test_models = X_test_e_model_names.values
test_input_shapes = X_test_e_input_shapes.values

In [51]:
print('Explained variance : ',explained_variance_score(y_test, y_pred))
print('R2_value : ',r2_score(y_test, y_pred))
print('Mean absolute error MAE :', mean_absolute_error(y_test, y_pred))
print('Root Mean squared error RMSE :', mean_squared_error(y_test, y_pred, squared=False))
print('Median absolute error MAD:', median_absolute_error(y_test, y_pred))

Explained variance :  0.9947118431944629
R2_value :  0.9945532486333553
Mean absolute error MAE : 5.238049919116064
Root Mean squared error RMSE : 7.721155603152515
Median absolute error MAD: 3.284628461083983


In [52]:
errors = abs(y_pred-np_test)
mape = 100 * (errors / np_test) 
accuracy = 100 - np.mean(mape)
print('MAX MAPE :', max(mape))
print('MAPE:', round(np.mean(mape), 2), '%.')
print('Accuracy:', round(accuracy, 2), '%.')

MAX MAPE : 44.243480376530584
MAPE: 8.28 %.
Accuracy: 91.72 %.


In [53]:
import csv
with open('./measured_vs_predicted_XGBoost_dim1.csv', 'w', newline='') as file : 
    writer = csv.writer(file)
    writer.writerow(['CNN_model', 'input_shape', 'model&shape', 'real_exec_time', 'predicted_exec_time', 'absolute_deviation', 'MAPE (in %)'])
    for i in range(0, len(y_pred)) :
        f = ''+test_models[i]+'_'+test_input_shapes[i]
        writer.writerow([test_models[i], test_input_shapes[i], f, np_test[i], y_pred[i],abs(y_pred[i]-np_test[i]), ((abs(y_pred[i]-np_test[i]))/np_test[i])*100])